In [1]:
import json
from requests import get, post, put

In [19]:
#Test Get Request
def getRequest (url):
    #URL = "http://127.0.0.1:5002/business"
    r = get(url=url, verify=False)
    dataRetrieved = r.json()
    return dataRetrieved

#Test Post Request
def postRequest (url):
    #url = 'http://127.0.0.1:5002/business'
    retrieve = post(url, json={'data':'remember'}).json()
    #Should retrieve json object
    return retrieve

def putRequst (url, inputJson):
    retrieve = put(url, json=inputJson).json()
    return retrieve

In [3]:
currentRecommendation = getRequest('http://127.0.0.1:5002/business')
print(currentRecommendation)

{'business_id': '4501', 'name': 'Lai Wah Heen', 'cuisine': 'Dim Sum', 'price': '$$$', 'rating': '3.5', 'distance': '1.0km', 'addText': '', 'explanation': 'best chicken dim-sum dumpling, good excellent service and many excellent fine chinese restaurant'}


In [4]:
#Just a sample
categoryList = ['Custom Cakes',
 'American (Traditional)', 'Food Court','Mexican','Italian','Food Tours','Wine Bars',
 'Imported Food','Food Delivery Services','Seafood','Brewpubs','Community Service/Non-Profit','Tea Rooms',
 'Caribbean','Fast Food','Shopping Centers','Gluten-Free','Tapas/Small Plates','Bangladeshi','Food Stands','Wine & Spirits',
 'Salvadoran','Champagne Bars','Pan Asian','Personal Chefs','Internet Cafes','Macarons','Taiwanese','Strip Clubs','Desserts',
 'Arts & Entertainment','Nicaraguan','Korean','Himalayan/Nepalese','Dinner Theater',
 'Japanese','Breakfast & Brunch','Performing Arts','Chinese','Southern','Tours','Waffles','Halal','Polish','Austrian','Kebab','Scottish','Cinema','South African',
 'Supper Clubs',
 'Sushi Bars','Bistros','Meat Shops','Musical Instruments & Teachers','Burgers','Malaysian','Cideries','Turkish',
 'Arcades','Afghan','Cheese Shops',
 'Cambodian','Indonesian','Bed & Breakfast','Golf','Festivals','Pakistani','Indian','Ramen',
 'Cantonese','Wineries','Chicken Shop','Pizza','Hakka','Mediterranean','Persian/Iranian','Ethiopian','Shaved Snow',
 'Specialty Schools','Caterers','Salad','Middle Eastern','Sandwiches','Specialty Food','Butcher','Cheesesteaks','Venezuelan',
 'Cannabis Clinics',
 'Shopping','Wedding Planning','Poke','Jazz & Blues','Irish Pub','Creperies','Vietnamese','Patisserie/Cake Shop',
 'Dive Bars',
 'Dance Clubs','Tobacco Shops','Pets','Art Galleries','Vegetarian','Chocolatiers & Shops','Coffee & Tea','French','Sri Lankan']

## Take in critique value, this step mimics the request sent by fulfillment critique_API_V1

In [118]:
#Construct the critiquing dictionary 
#categories
#restaurant -> boolean value, anything another than none should be taken
#price
#rating
#distance
#keywords
#critiqueValue
#positiveOrNegative

maxDistance = 169 
while True:
    satisfied = 'None'
    feature = 'None'
    positiveOrNegative = 'None'
    critiqueValue = 'None'
    critiqueDict = {}
    
    currentRecommendation = getRequest('http://127.0.0.1:5002/business')
    print('currentRecommendation:', currentRecommendation)
    
    #currentCuisineType = currentRecommendation['cuisine']
    #print('Recommending...')
    #displayRestaurantInfo(recommendIndex, current_user_item_predict,df,ID_dictionary)
    
    #Current recommendation cuisine type 
    #currentCuisineType = df[df['business_num_id'] == current_user_item_predict[recommendIndex]]\
    #                                                    .categories.unique()[0].split(', ')
    #currentPriceLabel = IP_dictionary[current_user_item_predict[recommendIndex]]
    #currentRating = IS_dictionary[current_user_item_predict[recommendIndex]]
    #currentDistance = ID_dictionary[current_user_item_predict[recommendIndex]]
    
    #First testing cuisine type
    print('\n????????????????')
    
    while satisfied.lower() not in ['yes', 'no', 'stop']: 
        satisfied = input("You Like? ('yes', 'no') ").strip().lower()
    
    if satisfied == 'stop' or satisfied == 'yes':
        print('BYE :)')
        break
        
    #When satisfied is NO, take in feature
    while feature.lower() not in ['name', 'cuisine', 'price', 'distance', 'rating', 'stop']: 
        feature = input("What feature to critique: (name, cuisine, price, rating, distance)").strip().lower()
    
    if feature == 'stop':
        break
        
    #Add value into dictionary 
    critiqueDict['feature'] = feature
    
    #Take in Positive or nagative 
    if feature != 'name':
        while positiveOrNegative.lower() not in ['positive', 'negative', 'stop']:
            positiveOrNegative = input("Positive or negative: ").strip().lower()
    
    if positiveOrNegative == 'stop':
        break
    
    #Add value into dictionary 
    critiqueDict['positiveOrNegative'] = positiveOrNegative
    
    #Only ask for critique value when not critiquing restaurant name, or not negatively critiquing price 
    if feature != 'name' and not(feature == 'price' and positiveOrNegative == 'negative')\
    and not(feature == 'distance' and positiveOrNegative == 'negative'):
    #and not(feature == 'rating' and positiveOrNegative == 'negative')\
     
        #The valid values to be critiuqed that can pass in 
        validCritiqueValueList = []
        #negatively critique current cuisine type, should only enter current cruisine type
        if 'cuisine' in feature and 'negative' in positiveOrNegative:
            validCritiqueValueList = currentRecommendation['cuisine'].lower().split(',') + ['stop']
            outPutString = '(' + currentRecommendation['cuisine'] +')'
        elif 'cuisine' in feature and 'positive' in positiveOrNegative:
            validCritiqueValueList = [cat.strip().lower() for cat in categoryList] + ['stop']
            #categories = list(set(IC_dictionary.keys())) + ['stop']
            outPutString = '(chinese, fast food,indian, ramen...)'
        #Can only enter cheapter or more expensive
        elif 'price' in feature:
            validCritiqueValueList = ['cheaper', 'more expensive','stop']
            outPutString = '(cheaper, more expensive)'
        elif 'rating' in feature:
            validCritiqueValueList = [str(star/10) for star in range(0,51,1)] + ['stop', 'current_rating']
            outPutString = '(0 ~ 5.0 with 0.1 increment)'
        #positively critique distance
        elif 'distance' in feature:
            validCritiqueDis = [str(i/10) for i in range(0,int((maxDistance+0.5)*10),5)][1:] 
            validCritiqueValueList = ['closer', 'further'] + validCritiqueDis + ['stop']
            outPutString = '(closer, further, or distance in range 0.5 ~' + str(maxDistance) + 'in every 0.5km)' 
        #Prompt to ask critique value
        while critiqueValue not in validCritiqueValueList:
            critiqueValue = input("Critique value: " + outPutString).strip().lower()
        
        if critiqueValue == 'stop':
            break
            
        #Add value into dictionary 
        critiqueDict['critiqueValue'] = critiqueValue
            
    print('????????????????\n')
    print('constructed json:', critiqueDict)
    
    try:
        retrieve = put('http://127.0.0.1:5002/business', json=critiqueDict).json()
        print(retrieve)
    
    except Exception as e:
        print(type(e))
        print(e)
    
    print('--------------------------------------------------------------')
    stopOrNot = input("Critique?").strip().lower()
    
    if stopOrNot == 'stop' or stopOrNot == 'no':
        print('BYE :)')
        break
    #At the end, send out the post request

currentRecommendation: {'business_id': '3', 'name': 'The Fry', 'cuisine': 'Bars, Korean, Soul Food', 'price': '$$', 'rating': '3.5', 'distance': '1.9km', 'addText': '', 'explanation': 'chicken is pretty good, pretty quick service and best korean chicken'}

????????????????
You Like? ('yes', 'no') no
What feature to critique: (name, cuisine, price, rating, distance)name
????????????????

constructed json: {'feature': 'name', 'positiveOrNegative': 'None'}
{'You_sent': {'feature': 'name', 'positiveOrNegative': 'None'}, 'Result': {'business_id': '3595', 'name': 'Lee Nam Jang', 'cuisine': 'Korean', 'price': '$$', 'rating': '3.5', 'distance': '2.0km', 'addText': 'Critiquing the current displayed item', 'explanation': 'super authentic korean experience, amazing soup and delicious yook-hwa raw beef'}}
--------------------------------------------------------------
Critique?yes
currentRecommendation: {'business_id': '3595', 'name': 'Lee Nam Jang', 'cuisine': 'Korean', 'price': '$$', 'rating': '3

## Clear up

In [74]:
getRequest('http://127.0.0.1:5002/clearup')

{'You are Resetting the user set up': 'Not retrieving anything for you'}

In [27]:
critiqueDict

{'feature': 'price', 'positiveOrNegative': 'negative'}

In [39]:
from time import sleep

In [48]:
try:
    retrieve = put('http://127.0.0./1:5003/business', json=critiqueDict).json()
    print(retrieve)
    
except Exception as e:
    print(type(e))
    print(e)

<class 'requests.exceptions.ConnectionError'>
HTTPConnectionPool(host='127.0.0.', port=80): Max retries exceeded with url: /1:5003/business (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E406EDC978>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


In [37]:
try:
    retrieve = put('http://127.0.0./1:5003/business', json=critiqueDict).json()
    print(retrieve)
    
except Exception as e:
    print(type(e))
    print(e)

<class 'requests.exceptions.ConnectionError'>
HTTPConnectionPool(host='127.0.0.', port=80): Max retries exceeded with url: /1:5003/business (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E406ED6DA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


In [8]:
postRequest('http://127.0.0.1:5002/business')

{'message': 'Internal Server Error'}

### Testing initialize & recommend for critique_API_V2

### Test user initialization - set up initial user recommendation list

In [114]:
restaurantInitializeDict = {
 'r_BrIgzYcwo1NAuG9dLbpg': 'Pai Northern Thai Kitchen',
 'aLcFhMe6DDJ430zelCpd2A': 'Khao San Road',
 'RtUvSWO_UZ8V3Wpj0n077w': 'KINKA IZAKAYA ORIGINAL',
 'iGEvDk6hsizigmXhDKs2Vg': 'Seven Lives Tacos Y Mariscos',
 'N93EYZy9R0sdlEvubu94ig': 'Banh Mi Boys',
 'zgQHtqX0gqMw1nlBZl2VnQ': 'Momofuku Noodle Bar',
 'B70iTJjcPkuYn8ouUewWgw': 'Gusto 101',
 'e41TP5cXZqSrz50xCBJqZw': 'Insomnia Restaurant & Lounge',
 'f5O7v_X_jCg2itqacRfxhg': 'Sansotei Ramen',
 'a8pmtlVKf7NiSLI-4KejIw': 'El Catrin Destileria',
 'h_4dPV9M9aYaBliH1Eoeeg': 'Wvrst',
 'kOFDVcnj-8fd3doIpCQ06A': "Mildred's Temple Kitchen",
 'Yl2TN9c23ZGLUBSD9ks5Uw': 'Byblos',
 'k6zmSLmYAquCpJGKNnTgSQ': 'The Stockyards',
 '_xAJZOKBMPOe47p1MphB2w': 'Fishman Lobster Clubhouse Restaurant',
 'MS-hfug4QDXqb_Mws3qlzA': 'Grand Electric',
 'O1TvPrgkK2bUo5O5aSZ7lw': 'Rol San',
 'nBl_4gw5ecGzNkHyzfii8g': 'Terroni',
 'KVpvE6pOPM9KMvak4HFsNg': 'Pizzeria Libretto',
 'fGurvC5BdOfd5MIuLUQYVA': 'Dumpling House Restaurant',
 'UxWH8zRYIBgs6Q2oykvRdw': 'Cluny Bistro & Boulangerie',
 'wSojc-y-d7MWiGWdy8deCg': 'Barque Smokehouse',
 'RUd_M7DPJq1I3DPq0oF--w': 'Lahore Tikka House',
 'MhiBpIBNTCAm1Xd3WzRzjQ': 'Messini Authentic Gyros',
 'oWTn2IzrprsRkPfULtjZtQ': "The Burger's Priest",
 'nHFJtud7jWZhM9dHQ1eIRA': 'Buk Chang Dong Soon Tofu',
 'Cp3YRVZojrCGeQS41Hf1pw': 'Pho Hung',
 'UDk7qUVIetK3Y3NuARguqg': 'Banjara Indian Cuisine',
 '3uIXuw878fNjxZ2Li1diFg': 'The Hogtown Vegan',
 'q5xrVJ4kivx_yEfJeOKNYQ': 'Bar Isabel'}

In [115]:
#Initialize user preference dictionary 'business_id': '5'
UserInitialResponse = {}
businessIdInput = ''
businessIdList = []
while businessIdInput.lower() != 'stop': 
    businessIdInput = input("Input business id that you like ").strip()
    print(businessIdInput)
    if businessIdInput== 'stop':
        break;
    elif businessIdInput not in restaurantInitializeDict.keys():
        print('invalid business id, RE-ENTER')
    else:
        businessIdList.append(businessIdInput)
        
UserInitialResponse['init_key'] = businessIdList
print(UserInitialResponse)
#Now retrieve from end point
retrieve = put('http://127.0.0.1:5002/initialization', json=UserInitialResponse, verify=False).json()
print(retrieve)


Input business id that you like nBl_4gw5ecGzNkHyzfii8g
nBl_4gw5ecGzNkHyzfii8g
Input business id that you like q5xrVJ4kivx_yEfJeOKNYQ
q5xrVJ4kivx_yEfJeOKNYQ
Input business id that you like UDk7qUVIetK3Y3NuARguqg
UDk7qUVIetK3Y3NuARguqg
Input business id that you like stop
stop
{'init_key': ['nBl_4gw5ecGzNkHyzfii8g', 'q5xrVJ4kivx_yEfJeOKNYQ', 'UDk7qUVIetK3Y3NuARguqg']}
{'You are calling a PUT for set up, the user initialization you sent:': {'init_key': ['nBl_4gw5ecGzNkHyzfii8g', 'q5xrVJ4kivx_yEfJeOKNYQ', 'UDk7qUVIetK3Y3NuARguqg']}}


### Test intersection - /initialization POST

In [116]:
yonge_and_finch = {'init_key': '43.779824,-79.415665'}
bloor_and_bathurst = {'init_key': '43.665194,-79.411208'}
queen_and_spadina = {'init_key': '43.648772,-79.396259'} 
bloor_and_yonge = {'init_key': '43.670409,-79.386814'}
dundas_and_yonge = {'init_key': '43.6561,-79.3802'}
spadina_and_dundas = {'init_key': '43.653004,-79.398082'}

intersectionDictionary= {'yonge_and_finch': yonge_and_finch,
                         'bloor_and_bathurst': bloor_and_bathurst,
                         'queen_and_spadina':queen_and_spadina,
                         'bloor_and_yonge': bloor_and_yonge,
                         'dundas_and_yonge': dundas_and_yonge,
                         'spadina_and_dundas': spadina_and_dundas}

In [117]:
testIntersection = ""
sendIntersectionJson = {}
while testIntersection  not in intersectionDictionary.keys():
    testIntersection = input('Input intersection to test')
    print(testIntersection)
    sendIntersectionJson['init_key'] = testIntersection
    print(sendIntersectionJson)
    
#Now retrieve from another end point
retrieve = post('http://127.0.0.1:5002/initialization', json=sendIntersectionJson, verify=False).json()
print(retrieve)

currentRecommendation = getRequest('http://127.0.0.1:5002/business')
print('currentRecommendation:', currentRecommendation)

Input intersection to testyonge_and_finch
yonge_and_finch
{'init_key': 'yonge_and_finch'}
{'You are calling a POST for set up, intersection point:': 'yonge_and_finch'}
currentRecommendation: {'business_id': '3', 'name': 'The Fry', 'cuisine': 'Bars, Korean, Soul Food', 'price': '$$', 'rating': '3.5', 'distance': '1.9km', 'addText': '', 'explanation': 'chicken is pretty good, pretty quick service and best korean chicken'}


In [1]:
def detect_intent_texts(project_id, session_id, texts, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    import dialogflow_v2 as dialogflow
    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    print('Session path: {}\n'.format(session))

    for text in texts:
        text_input = dialogflow.types.TextInput(
            text=text, language_code=language_code)

        query_input = dialogflow.types.QueryInput(text=text_input)

        response = session_client.detect_intent(
            session=session, query_input=query_input)

        print('=' * 20)
        print('Query text: {}'.format(response.query_result.query_text))
        print('Detected intent: {} (confidence: {})\n'.format(
            response.query_result.intent.display_name,
            response.query_result.intent_detection_confidence))
        print('Fulfillment text: {}\n'.format(
            response.query_result.fulfillment_text))

In [ ]:
detect_intent_texts()